Importando as bibliotecas que usaremos

In [1]:
import pandas as pd
import math
import numpy as np
import googlemaps as maps
import plotly.express as px
import geopandas as gpd
import folium
from folium import Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

Abrindo o arquivo excel:

In [2]:
df = pd.read_excel('C:/GitHub/Ci-ncia-de-Dados/VI Desafio Ciência de Dados - Puc Goiás/acidentesTransito_cenipa_2010a2021.xlsx')
print(df.head())

  ocorrencia_classificacao  ocorrencia_latitude  ocorrencia_longitude  \
0          INCIDENTE GRAVE            -9.868333            -67.898056   
1                 ACIDENTE           -12.204167            -56.237778   
2                 ACIDENTE           -12.204167            -56.237778   
3                 ACIDENTE           -12.204167            -56.237778   
4                 ACIDENTE           -12.204167            -56.237778   

   ocorrencia_cidade ocorrencia_uf ocorrencia_pais ocorrencia_aerodromo  \
0         RIO BRANCO            AC          BRASIL                 SBRB   
1  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
2  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
3  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
4  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   

  ocorrencia_dia     ocorrencia_hora investigacao_aeronave_liberada  ...  \
0     2022-04-14 1900-01-01 17:45:

Tratamento dos dados dentro do data frame:

In [3]:
## Buscando e removendo dados duplicados, caso haja
df.drop_duplicates()

# verificando quantas coordenas nulas eu tenho
print(df.isnull().sum())

ocorrencia_classificacao              0
ocorrencia_latitude                   0
ocorrencia_longitude                  0
ocorrencia_cidade                     0
ocorrencia_uf                         0
ocorrencia_pais                       0
ocorrencia_aerodromo                  0
ocorrencia_dia                        0
ocorrencia_hora                       0
investigacao_aeronave_liberada     1546
investigacao_status                   0
divulgacao_relatorio_numero          10
divulgacao_relatorio_publicado        0
divulgacao_dia_publicacao             0
total_recomendacoes                   0
total_aeronaves_envolvidas            0
ocorrencia_saida_pista                0
ocorrencia_tipo                       4
ocorrencia_tipo_categoria             4
taxonomia_tipo_icao                   4
recomendacao_numero                   0
recomendacao_dia_assinatura           0
recomendacao_dia_encaminhamento       0
recomendacao_dia_feedback          1106
recomendacao_conteudo                 0


Remover onde a latitude ou longitude são nulas, não quero elas


In [4]:
df.dropna(subset=['ocorrencia_latitude'], inplace=True)
df.dropna(subset=['ocorrencia_longitude'], inplace=True)
print(df.isnull().sum())

ocorrencia_classificacao              0
ocorrencia_latitude                   0
ocorrencia_longitude                  0
ocorrencia_cidade                     0
ocorrencia_uf                         0
ocorrencia_pais                       0
ocorrencia_aerodromo                  0
ocorrencia_dia                        0
ocorrencia_hora                       0
investigacao_aeronave_liberada     1546
investigacao_status                   0
divulgacao_relatorio_numero          10
divulgacao_relatorio_publicado        0
divulgacao_dia_publicacao             0
total_recomendacoes                   0
total_aeronaves_envolvidas            0
ocorrencia_saida_pista                0
ocorrencia_tipo                       4
ocorrencia_tipo_categoria             4
taxonomia_tipo_icao                   4
recomendacao_numero                   0
recomendacao_dia_assinatura           0
recomendacao_dia_encaminhamento       0
recomendacao_dia_feedback          1106
recomendacao_conteudo                 0


Passar para numerico, caso haja algum valor que não esteja numerico

In [5]:
df['ocorrencia_latitude'] = pd.to_numeric(df['ocorrencia_latitude'])
df['ocorrencia_longitude'] = pd.to_numeric(df['ocorrencia_longitude'])


Formatar as coordenadas

In [7]:
estados_brasileiros = {
    "AC": (-9.115833, -67.784167),
    "AL": (-9.616667, -35.733333),
    "AP": (1.45, -51.75),
    "AM": (-3.100833, -60.025),
    "BA": (-12.971111, -38.477778),
    "CE": (-3.716667, -38.541667),
    "DF": (-15.794444, -47.882222),
    "ES": (-19.926111, -40.302778),
    "GO": (-15.981667, -49.253056),
    "MA": (-2.533333, -44.25),
    "MT": (-12.65, -55.45),
    "MS": (-20.448611, -54.627222),
    "MG": (-19.926111, -43.937222),
    "PA": (-1.458056, -48.483333),
    "PB": (-7.109444, -35.874444),
    "PR": (-25.428889, -51.521111),
    "PE": (-8.058333, -34.879167),
    "PI": (-7.216667, -41.766667),
    "RJ": (-22.908333, -43.196111),
    "RN": (-5.808889, -35.201111),
    "RS": (-30.033333, -51.216667),
    "RO": (-10.748889, -62.658056),
    "RR": (2.816667, -60.675),
    "SC": (-27.594444, -48.541667),
    "SP": (-23.550556, -46.633333),
    "SE": (-10.916667, -37.066667),
    "TO": (-10.183333, -48.325)
}


Atualizando a coluna da longitude, com os valores formadados.

In [8]:
for index, row in df['ocorrencia_longitude'].items():

    if not np.isnan(row):
        log_base_10 = np.log10(abs(row))
        divisor = math.ceil(log_base_10) - 2
        if divisor != 0:
            df.loc[index, 'ocorrencia_longitude'] = row / (10**divisor)
        #print(index, row, df.loc[index, 'ocorrencia_longitude']) 
            
df.to_excel('acidentesTransito_cenipa_2010a2021.xlsx', index=False)


Atualizando a coluna da latitude, com os valores formatados.

In [9]:
for index, row in df['ocorrencia_latitude'].items():
    if not np.isnan(row):
        log_base_10 = np.log10(abs(row))
        divisor = math.ceil(log_base_10)
        if(row > 0):
            divisor -= 1
            df.loc[index, 'ocorrencia_latitude'] = row / (10 ** divisor)
        else:
            estado = df['ocorrencia_uf'].iloc[index]  # Converte a Series em uma string
            
            divisor -= 2
            value = row/(10**divisor)
            if(abs(value - estados_brasileiros[estado][0]) > 10):
                value /= 10
            df.loc[index, 'ocorrencia_latitude'] = value

df.to_excel('acidentesTransito_cenipa_2010a2021.xlsx', index=False)

list = df.sort_values(by='ocorrencia_latitude')           
print(list)


     ocorrencia_classificacao  ocorrencia_latitude  ocorrencia_longitude  \
3581                 ACIDENTE           -33.384722            -53.254444   
3579                 ACIDENTE           -33.384722            -53.254444   
3580                 ACIDENTE           -33.384722            -53.254444   
413                  ACIDENTE           -33.176944            -53.014167   
412                  ACIDENTE           -33.176944            -53.014167   
...                       ...                  ...                   ...   
3899                 ACIDENTE             5.084167            -60.345556   
3897                 ACIDENTE             5.084167            -60.345556   
3906                 ACIDENTE             5.084167            -60.345556   
3908                 ACIDENTE             5.084167            -60.345556   
4437          INCIDENTE GRAVE             8.713611             63.902778   

            ocorrencia_cidade ocorrencia_uf ocorrencia_pais  \
3581  SANTA VITÓRIA DO P

Mapa com as coordenadas

In [10]:
import folium


# Extraindo as colunas de latitude e longitude
latitudes = df['ocorrencia_latitude']
longitudes = df['ocorrencia_longitude']

# Criando o mapa
mapa_mundo = folium.Map(location=[0, 0], zoom_start=2, tiles ='cartodbpositron')

# Adicionando marcadores
for lat, lon in zip(latitudes, longitudes):
  folium.Marker([lat, lon], popup=f"Incidente").add_to(mapa_mundo)


# Salvando o mapa
mapa_mundo.save('mapa_mundo.html')

#mapa_mundo
